In [1]:
# imports
import pandas as pd
import numpy as np
import time

In [2]:
# reading in csv
intake_df = pd.read_csv('../cleanedSources/intakeCleaned.csv')

In [3]:
# examining df
intake_df.head()

,id,name_intake,dateTime_intake,loc_found,intake_type,intake_condition,animal_type,sex_intake,age_intake,breed,colour
0,A786884,Brock,2019-01-03 16:19:00,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor
1,A706918,Belle,2015-07-05 12:59:00,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver
2,A724273,Runster,2016-04-14 18:43:00,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White
3,A665644,Unknown,2013-10-21 07:59:00,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico
4,A682524,Rio,2014-06-29 10:38:00,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray


In [4]:
# observing ages present in df
intake_df.groupby(['age_intake']).count()

,id,name_intake,dateTime_intake,loc_found,intake_type,intake_condition,animal_type,sex_intake,breed,colour
age_intake,,,,,,,,,,
-1 years,6,6,6,6,6,6,6,6,6,6
-2 years,1,1,1,1,1,1,1,1,1,1
-3 years,1,1,1,1,1,1,1,1,1,1
0 years,824,824,824,824,824,824,824,824,824,824
1 day,677,677,677,677,677,677,677,677,677,677
1 month,12531,12531,12531,12531,12531,12531,12531,12531,12531,12531
1 week,1076,1076,1076,1076,1076,1076,1076,1076,1076,1076
1 weeks,951,951,951,951,951,951,951,951,951,951
1 year,22383,22382,22383,22383,22383,22383,22383,22383,22383,22383


In [5]:
# removing negative values, assuming the negatives are transcription errors
for x in range (0, len(intake_df)):
    if intake_df.iloc[x, 8][0] == '-':
        intake_df.iloc[x, 8] = intake_df.iloc[x, 8][1:]

In [6]:
# creating ages in months and years for better consistency using a function and vectorisation
def norm_age(col, newcol1, newcol2):
    months_list = []
    years_list = []
    # for loop to iterate through df
    for x in range (0, len(intake_df)):
        # split each value in the age_intake col by spaces
        # if it ends with year/years, multiply by 12 and enter into month col; enter number into year col
        if (intake_df.iloc[x, col].split(' ')[1] == 'year') or (intake_df.iloc[x, col].split(' ')[1] == 'years'):
            age_val = int(intake_df.iloc[x, col].split(' ')[0])
            months_list.append(round(age_val * 12, 2))
            years_list.append(round(age_val, 2))
        # if it ends with month/months, enter number into month col; divide by 12 and place into year col
        elif (intake_df.iloc[x, col].split(' ')[1] == 'month') or (intake_df.iloc[x, col].split(' ')[1] == 'months'):
            age_val = int(intake_df.iloc[x, col].split(' ')[0])
            months_list.append(round(age_val, 2))
            years_list.append(round(age_val/12, 2))
        # if it ends with week/weeks, divide by four and place into month col; leave default value in year col
        elif (intake_df.iloc[x, col].split(' ')[1] == 'week') or (intake_df.iloc[x, col].split(' ')[1] == 'weeks'):
            age_val = int(intake_df.iloc[x, col].split(' ')[0])
            months_list.append(round(age_val/4, 2))
            years_list.append(0)
        # else (if it ends with day or days), leave default value in both cols
        else:
            months_list.append(0)
            years_list.append(0)
    intake_df[f'{newcol1}'] = months_list
    intake_df[f'{newcol2}'] = years_list

In [7]:
# vectorising and running the funct (using different col names to examine side by side)
vectfunc = np.vectorize(norm_age)
vectfunc(8, 'age_intake_months', 'age_intake_years')
intake_df.head()

,id,name_intake,dateTime_intake,loc_found,intake_type,intake_condition,animal_type,sex_intake,age_intake,breed,colour,age_intake_months,age_intake_years
0,A786884,Brock,2019-01-03 16:19:00,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor,24.0,2.00
1,A706918,Belle,2015-07-05 12:59:00,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver,96.0,8.00
2,A724273,Runster,2016-04-14 18:43:00,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White,11.0,0.92
3,A665644,Unknown,2013-10-21 07:59:00,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico,1.0,0.00
4,A682524,Rio,2014-06-29 10:38:00,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,48.0,4.00


In [8]:
# re-observing ages present in df (months)
intake_df.groupby(['age_intake_months']).count()

,id,name_intake,dateTime_intake,loc_found,intake_type,intake_condition,animal_type,sex_intake,age_intake,breed,colour,age_intake_years
age_intake_months,,,,,,,,,,,,
0.00,3492,3492,3492,3492,3492,3492,3492,3492,3492,3492,3492,3492
0.25,2027,2027,2027,2027,2027,2027,2027,2027,2027,2027,2027,2027
0.50,2638,2638,2638,2638,2638,2638,2638,2638,2638,2638,2638,2638
0.75,3801,3801,3801,3801,3801,3801,3801,3801,3801,3801,3801,3801
1.00,17209,17208,17209,17209,17209,17209,17209,17209,17209,17209,17209,17209
1.25,328,328,328,328,328,328,328,328,328,328,328,328
2.00,7071,7071,7071,7071,7071,7071,7071,7071,7071,7071,7071,7071
3.00,3395,3395,3395,3395,3395,3395,3395,3395,3395,3395,3395,3395
4.00,3296,3296,3296,3296,3296,3296,3296,3296,3296,3296,3296,3296


In [9]:
# re-observing ages present in df (years)
intake_df.groupby(['age_intake_years']).count()

,id,name_intake,dateTime_intake,loc_found,intake_type,intake_condition,animal_type,sex_intake,age_intake,breed,colour,age_intake_months
age_intake_years,,,,,,,,,,,,
0.00,16964,16963,16964,16964,16964,16964,16964,16964,16964,16964,16964,16964
0.08,12531,12531,12531,12531,12531,12531,12531,12531,12531,12531,12531,12531
0.17,7071,7071,7071,7071,7071,7071,7071,7071,7071,7071,7071,7071
0.25,3395,3395,3395,3395,3395,3395,3395,3395,3395,3395,3395,3395
0.33,3296,3296,3296,3296,3296,3296,3296,3296,3296,3296,3296,3296
0.42,3171,3171,3171,3171,3171,3171,3171,3171,3171,3171,3171,3171
0.50,2441,2441,2441,2441,2441,2441,2441,2441,2441,2441,2441,2441
0.58,1912,1911,1912,1912,1912,1912,1912,1912,1912,1912,1912,1912
0.67,1520,1520,1520,1520,1520,1520,1520,1520,1520,1520,1520,1520


In [12]:
# re-saving to csv
intake_df.to_csv('../cleanedSources/intakeCleaned.csv', index=False)

In [10]:
# checking time it takes to run the vector function
%timeit vectfunc(8, 'age_intake_months', 'age_intake_years')

32.5 s ± 2.31 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
# trying age regularisation through for loop for time comparison
t1=time.time()
months_list = []
years_list = []
# for loop to iterate through df
for x in range (0, len(intake_df)):
    # split each value in the age_intake col by spaces
    # if it ends with year/years, multiply by 12 and enter into month col; enter number into year col
    if (intake_df.iloc[x, 8].split(' ')[1] == 'year') or (intake_df.iloc[x, 8].split(' ')[1] == 'years'):
        age_val = int(intake_df.iloc[x, 8].split(' ')[0])
        months_list.append(age_val * 12)
        years_list.append(age_val)
    # if it ends with month/months, enter number into month col; divide by 12 and place into year col
    elif (intake_df.iloc[x, 8].split(' ')[1] == 'month') or (intake_df.iloc[x, 8].split(' ')[1] == 'months'):
        age_val = int(intake_df.iloc[x, 8].split(' ')[0])
        months_list.append(age_val)
        years_list.append(age_val/12)
    # if it ends with week/weeks, divide by four and place into month col; leave default value in year col
    elif (intake_df.iloc[x, 8].split(' ')[1] == 'week') or (intake_df.iloc[x, 8].split(' ')[1] == 'weeks'):
        age_val = int(intake_df.iloc[x, 8].split(' ')[0])
        months_list.append(age_val/4)
        years_list.append(0)
    # else (if it ends with day or days), leave default value in both cols
    else:
        months_list.append(0)
        years_list.append(0)
t2=time.time()
print("\nTime taken by for-loop\n----------------------------------------------\n{} us".format(1000000*(t2-t1)))
# adding values to cols
# intake_df['age_intake_months'] = months_list
# intake_df['age_intake_years'] = years_list
# # examining results
# intake_df.head()


Time taken by for-loop
----------------------------------------------
14825141.906738281 us
